In [2]:
import pandas as pd
import altair as alt

In [55]:
kof_df = pd.read_csv('Datasets/KOF_clean.csv')
kof_df.head()

,code,country,year,KOFGI,KOFGIdf,KOFGIdj,KOFEcGI,KOFEcGIdf,KOFEcGIdj,KOFTrGI,...,KOFIpGIdj,KOFInGI,KOFInGIdf,KOFInGIdj,KOFCuGI,KOFCuGIdf,KOFCuGIdj,KOFPoGI,KOFPoGIdf,KOFPoGIdj
0,BRA,Brazil,1990,42.0,44.0,40.0,28.0,24.0,31.0,26.0,...,32.0,39.0,32.0,46.0,40.0,29.0,51.0,65.0,85.0,46.0
1,BRA,Brazil,1991,44.0,43.0,44.0,27.0,23.0,31.0,27.0,...,32.0,39.0,31.0,46.0,38.0,30.0,46.0,71.0,84.0,58.0
2,BRA,Brazil,1992,45.0,45.0,45.0,29.0,27.0,32.0,28.0,...,33.0,40.0,31.0,50.0,38.0,29.0,47.0,72.0,84.0,60.0
3,BRA,Brazil,1993,46.0,46.0,46.0,30.0,29.0,32.0,29.0,...,32.0,41.0,30.0,52.0,40.0,32.0,47.0,73.0,84.0,62.0
4,BRA,Brazil,1994,48.0,45.0,50.0,29.0,25.0,33.0,29.0,...,33.0,41.0,30.0,52.0,39.0,34.0,44.0,81.0,86.0,75.0


In [56]:
dataset_list = ['BOVESPA-BRA-Clean', 'CAC40-FRA-Clean', 'DAX30-DEU-Clean', 'HangSeng-HKG-Clean', 'Nikkei225-JPN-Clean', 'Shanghai-CHN-Clean']

countries_df = pd.DataFrame(columns=['year', 'YoY Change', 'code'])
for dataset in dataset_list:
    df = pd.read_csv('./Datasets/' + dataset + '.csv')
    countries_df = pd.concat([countries_df, df])


kof_df = kof_df.merge(countries_df, on=['year', 'code'])

In [57]:
kof_df

,code,country,year,KOFGI,KOFGIdf,KOFGIdj,KOFEcGI,KOFEcGIdf,KOFEcGIdj,KOFTrGI,...,KOFInGI,KOFInGIdf,KOFInGIdj,KOFCuGI,KOFCuGIdf,KOFCuGIdj,KOFPoGI,KOFPoGIdf,KOFPoGIdj,YoY Change
0,BRA,Brazil,1995,49.0,45.0,54.0,30.0,24.0,36.0,30.0,...,43.0,31.0,55.0,40.0,37.0,44.0,82.0,84.0,80.0,2.071614
1,BRA,Brazil,1996,50.0,46.0,55.0,31.0,25.0,37.0,29.0,...,46.0,31.0,60.0,41.0,37.0,44.0,84.0,87.0,81.0,58.436023
2,BRA,Brazil,1997,52.0,48.0,56.0,32.0,26.0,38.0,29.0,...,53.0,44.0,63.0,42.0,38.0,45.0,83.0,86.0,81.0,50.672130
3,BRA,Brazil,1998,52.0,48.0,57.0,34.0,27.0,40.0,28.0,...,54.0,45.0,63.0,43.0,40.0,45.0,83.0,84.0,82.0,-33.759542
4,BRA,Brazil,1999,54.0,50.0,58.0,37.0,33.0,41.0,28.0,...,55.0,47.0,63.0,44.0,42.0,46.0,83.0,84.0,83.0,143.877845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,JPN,Japan,2016,78.0,72.0,84.0,65.0,51.0,79.0,55.0,...,93.0,93.0,92.0,91.0,86.0,95.0,88.0,90.0,85.0,6.195768
168,JPN,Japan,2017,78.0,73.0,84.0,67.0,54.0,80.0,56.0,...,93.0,93.0,92.0,91.0,87.0,95.0,87.0,89.0,86.0,19.966000
169,JPN,Japan,2018,78.0,72.0,84.0,67.0,55.0,80.0,57.0,...,93.0,93.0,93.0,90.0,85.0,95.0,87.0,88.0,86.0,-16.780033
170,JPN,Japan,2019,76.0,68.0,84.0,68.0,55.0,80.0,57.0,...,93.0,94.0,93.0,92.0,88.0,95.0,79.0,72.0,86.0,18.622367


In [58]:
alt.Chart(kof_df).mark_point().encode(
    x='KOFGI',
    y='YoY Change:Q'
)

alt.Chart(...)